<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Alina/Aufgabe6/Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install box2d-py

     |████████████████████████████████| 450kB 5.2MB/s 


In [3]:
import gym
import random
import torch
import numpy as np
from collections import namedtuple, deque
import matplotlib.pyplot as plt
%matplotlib inline

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Initialisieren der Umgebung



In [4]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of Actions: ', env.action_space.n)

State shape:  (8,)
Number of Actions:  4


1. Define a neural network with two fully connected-layers. The hidden layer uses a
Relu activation. The output layer uses a softmax. Try different hidden layer sizes
(between 100 and 500). The network takes as input a vector of the current states
and gives out a probability for each action.

In [5]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, state_size, hidden_layer, action_size):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(state_size, hidden_layer)
        self.linear2 = torch.nn.Linear(hidden_layer, action_size)

    def forward(self, state):
        #Transformiere state (numpay array) zu Torch Tensor, da TwoLayerNet Tensor als Input erwartet
        state = torch.from_numpy(state)
        h_relu = F.relu(self.linear1(state))
        action_pred = F.softmax(self.linear2(h_relu))
        return action_pred

2. Generate 100 episodes by sampling actions using the network output. Limit the
number of steps per episode to 500. Sum up the reward of all steps of one episode.

In [6]:
def play_episodes(max_steps, model): 

    t = 0
    done = False
    action_space = [0, 1, 2, 3]

    #State-Action-Pairs pro Zeitschritt einer Episode
    state_action_per_timestep = []
    #Reward pro Zeitschritt einer Episode
    reward_per_timestep = []
     
    state = env.reset() #State ist ein 1-D Numpy Array mit 8 Einträgen 

    #Erstellung einer Episode
    while t < max_steps: 

      #print("STATE:::::::: ", state)
      #print("STATE TYPE + SHAPE ::::::::::::", state.shape, type(state) )
        
      action_output = model(state)
      #print("ACTIONS OUTPUT::::::::::::::", action_output)
      sampled_action = random.choices(action_space, action_output, k=1)
      #print("CHOOSEN ACTION:::::::::.", sampled_action[0])

      state_action_per_timestep.append((state, sampled_action[0]))

      next_state, reward, done, _ = env.step(sampled_action[0])
      reward_per_timestep.append(reward)

      state = next_state
      t += 1
     
      if done: 
        break
    
    return state_action_per_timestep, reward_per_timestep, t

In [7]:
episode_n = 100
max_steps = 500

#Rewards aller n Episoden
rewards_per_episode = []
#Durchschnittlicher Reward einer Episode
mean_rewards_per_episode = []

state_size, action_size, hidden_layer = 8, 64, 4
model = TwoLayerNet(state_size, action_size, hidden_layer)

for i in range(0, episode_n):
  state_actions, reward, episode_length = play_episodes(max_steps, model)
 
  #reward_per_episode = Liste mit allen summierten Rewards von 100 Episoden 
  rewards_per_episode.append((sum(reward), state_actions))
  mean_rewards_per_episode.append((sum(reward)/episode_length, state_actions))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


3. Print out the mean reward per episode of the 100 episodes.

In [8]:
#mean_rewards_per_episode

4. Identify the 20 best of those episodes in terms of reward and use the
<state, action> pairs of these episodes as training examples for the network.

In [9]:
#Sortiere Rewards absteigend
reward_sorted = sorted(rewards_per_episode)
mean_rewards_sorted = sorted(mean_rewards_per_episode)

#Schneide die besten 20 ab und extrahiere die State,actions pairs dieser Episoden 
top20 = reward_sorted[:20]
training_pairs = [x[1] for x in top20]

#print(training_pairs)

In [10]:


for pair in training_pairs:
  print(pair[0][0])
  #print(pair[1])


[-5.9156417e-04  1.4134574e+00 -5.9935719e-02  1.1277095e-01
  6.9228926e-04  1.3576316e-02  0.0000000e+00  0.0000000e+00]
[ 0.00522709  1.4177614   0.5294286   0.30404913 -0.00605008 -0.11992346
  0.          0.        ]
[ 0.0065505   1.4197313   0.6634812   0.39158756 -0.00758363 -0.15028855
  0.          0.        ]
[ 0.006989    1.4177092   0.7079054   0.30172586 -0.00809183 -0.16035125
  0.          0.        ]
[-0.00163336  1.4191597  -0.16546687  0.36620596  0.00189956  0.03748075
  0.          0.        ]
[-0.00542288  1.4223585  -0.54929304  0.50836444  0.00629053  0.12442312
  0.          0.        ]
[ 1.1904717e-03  1.4175390e+00  1.2057171e-01  2.9417294e-01
 -1.3727234e-03 -2.7311290e-02  0.0000000e+00  0.0000000e+00]
[-0.0078969   1.4192797  -0.7998873   0.37151334  0.00915733  0.18118647
  0.          0.        ]
[ 0.00588312  1.4205422   0.5958686   0.42763907 -0.00681016 -0.13497312
  0.          0.        ]
[-8.6011889e-04  1.4090264e+00 -8.7149598e-02 -8.4163390e-02


In [11]:
def train(training_set, model, optimizer):

  #Optimizer 
  
  model.train()
  for pair in training_set:

    state = pair[0][0]
    target_action = pair[0][1]

    target = torch.zeros(1, dtype=torch.long)
    target[0] = target_action

    output_action = model(state).unsqueeze(0)
    #print(f"TARGET ACTIOM {target.shape}")
    #print(f"TARGET ACTIOM {output_action.shape}")

    loss = F.nll_loss(output_action, target)
    loss.backward()
    optimizer.step()

In [13]:
#train(training_pairs, model)

In [16]:
mean_reward = 0 # as Learning Indicator
episode_n = 100
max_steps = 500
optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.1)

#Definiere das Neuronale Netz mit einer Hidden_Layer
state_size, hidden_layer, action_size = 8, 64, 4
model = TwoLayerNet(state_size, hidden_layer, action_size)

while mean_reward < 100: 

  #2: Generate 100 episodes by sampling actions using the network output
  
  #Rewards aller n Episoden
  rewards_per_episode = []
  #Durchschnittlicher Reward einer Episode
  mean_rewards_per_episode = []

  for i in range(0, episode_n):
    state_actions, reward, episode_length = play_episodes(max_steps, model)
  
    mean_reward = sum(reward) / episode_length
    sum_reward = sum(reward) 
    
    #reward_per_episode = Liste mit allen summierten Rewards von 100 Episoden
    rewards_per_episode.append((sum_reward, state_actions))
    mean_rewards_per_episode.append((mean_reward, state_actions))

    #3: Print out the mean reward per episode of the 100 episodes.
    print(f"Mean Reward in Episode {i}: {mean_reward}")
  
  #4: Identify the 20 best of those episodes in terms of reward and use the <state, action> pairs of these episodes as training examples for the network.
  #Sortiere Rewards absteigend
  reward_sorted = sorted(rewards_per_episode)
  
  #Schneide die besten 20 ab und extrahiere die State,actions pairs dieser Episoden 
  top20 = reward_sorted[:20]
  training_pairs = [x[1] for x in top20]

  train(training_pairs, model, optimizer)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


Mean Reward in Episode 0: -1.467336054100702
Mean Reward in Episode 1: -2.1465715496049214
Mean Reward in Episode 2: -3.5341887978665323
Mean Reward in Episode 3: -3.678627881193579
Mean Reward in Episode 4: -0.9149076247969989
Mean Reward in Episode 5: -1.0572783985718566
Mean Reward in Episode 6: -1.7088110898172275
Mean Reward in Episode 7: -0.4823639071207926
Mean Reward in Episode 8: -1.252849972784542
Mean Reward in Episode 9: -1.3690719852500013
Mean Reward in Episode 10: -1.9247901810770152
Mean Reward in Episode 11: -1.660934389492077
Mean Reward in Episode 12: -0.4348224572240298
Mean Reward in Episode 13: -1.4963954661724275
Mean Reward in Episode 14: -3.2495544204450657
Mean Reward in Episode 15: -0.9979129493608219
Mean Reward in Episode 16: -1.4832629586017843
Mean Reward in Episode 17: -4.599549684859726
Mean Reward in Episode 18: -1.9246355440816838
Mean Reward in Episode 19: -1.242579214814068
Mean Reward in Episode 20: -1.2197609220690588
Mean Reward in Episode 21: -1

KeyboardInterrupt: ignored

In [ ]:
test = []
test.append(([0,2,3,],1))

np.array(test)